# RNN实现语言模型

## 1. 下载PTB文本数据集

数据下载地址为： `http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz`
在这里，解压后只使用`data/`文件夹内的下面三个文件：
```
ptb.test.txt
ptb.train.txt
ptb.valid.txt
```

## 2.  构建模型

In [1]:
import numpy as np
import tensorflow as tf
import ptb.reader as reader

In [2]:
DATA_PATH = './data/simple-examples/data/'
HIDDEN_SIZE = 200
NUM_LAYERS = 2
VOCAB_SIZE = 10000
EMBED_SIZE = HIDDEN_SIZE

LEARNING_RATE = 1.0
TRAIN_BATCH_SIZE = 20
TRAIN_NUM_STEP = 35

EVAL_BATCH_SIZE = 1
EVAL_NUM_STEP = 1
NUM_EPOCH = 2
KEEP_PROB = 0.5
MAX_GRAD_NORM = 5

In [3]:
# 模型描述
class PTBModel(object):
    def __init__(self, is_training, batch_size, num_steps):
        self.batch_size = batch_size
        self.num_steps = num_steps
        
        self.input_data = tf.placeholder(tf.int32, [batch_size, num_steps])
        self.targets = tf.placeholder(tf.int32, [batch_size, num_steps])
        
        lstm_cell = tf.contrib.rnn.BasicLSTMCell(HIDDEN_SIZE)
        if is_training:
            lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell,  output_keep_prob=KEEP_PROB)
        cell = tf.contrib.rnn.MultiRNNCell([lstm_cell]* NUM_LAYERS)        
        self.initial_state = cell.zero_state(batch_size, tf.float32)
        # ??EMBED_SIZE ==HIDDEN_SIZE?
        embedding = tf.get_variable("embedding", [VOCAB_SIZE, EMBED_SIZE])  
        inputs = tf.nn.embedding_lookup(embedding, self.input_data)
        if is_training:
            inputs = tf.nn.dropout(inputs, KEEP_PROB)
            
        outputs = []
        state = self.initial_state
        with tf.variable_scope("RNN"):
            for time_step in range(num_steps):
                if time_step >0:
                    tf.get_variable_scope().reuse_variables()
                cell_output, state = cell(inputs[:, time_step,:], state)
                outputs.append(cell_output)
        output = tf.reshape(tf.concat(1, outputs), [-1, HIDDEN_SIZE_SIZE])
        
        weight = tf.get_variable("weight", [HIDDEN_SIZE, VOCAB_SIZE])
        bias = tf.get_variable("bias", [VOCAB_SIZE])
        logits = tf.matmul(output, weight) + bias
        
        loss = tf.nn.seq2seq.sequence_loss_by_example([logits], 
                                                      [tf.reshape(self.targets, [-1])],
                                                      [tf.ones([batch_size*num_step], dtype=tf.float32)] )
        
        self.cost = tf.reduce_sum(loss)/batch_size
        self.final_state = state
        
        if not is_training:
            return
        
        trainable_variables = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, trainable_variables), MAX_GRAD_NORM)
        optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE)
        self.train_op = optimizer.apply_gradients(zip(grads, trainable_variables))

In [4]:
def run_epoch(session, model, data, train_op, output_log):
    total_costs = 0.0
    iters = 0
    state = session.run(model.initial_state)
    for step, (x, y) in enumearate(reader.ptb_iterator(data, model.batch_size, model.num_steps)):
        cost, state, _ = session.run([model.cost, model.final_state, train_op], {model.input_data:x, model.targets:y, model.initial_state:state})
        total_costs += cost
        iters += model.num_steps
    return np.exp(total_costs/iters)

In [5]:
train_data, valid_data, test_data, _ = reader.ptb_raw_data(DATA_PATH)

initializer = tf.random_uniform_initializer(-0.05, 0.05)
with tf.variable_scope("language_model", reuse=None, initializer=initializer):
    train_model = PTBModel(True, TRAIN_BATCH_SIZE, TRAIN_NUM_STEP)
with tf.variable_scope("language_model", reuse=True, initializer=initializer):
    eval_model = PTBModel(False, EVAL_BATCH_SIZE, EVAL_NUM_STEP)

TypeError: Expected int32, got list containing Tensors of type '_Message' instead.